# Script to analyze GitHub trend data and upload results to Dynamo

In [125]:
# VC Sourcing Pipeline - Complete Setup and Configuration
import aiohttp  # type: ignore
import asyncio
import os
import math
from typing import Dict, List, Optional, TypeGuard
from dotenv import load_dotenv  # type: ignore
from openai import AsyncOpenAI, ChatCompletion  # type: ignore
from pydantic import BaseModel, Field  # type: ignore
from exa_py import Exa  # type: ignore
from github import Github, UnknownObjectException  # type: ignore
from github.Repository import Repository  # type: ignore

# Load environment variables
load_dotenv()

# API Configuration
BASE_OSS_INSIGHT_URL = "https://api.ossinsight.io/v1"
TRENDING_REPOS_URL = f"{BASE_OSS_INSIGHT_URL}/trends/repos"
COLLECTIONS_URL = f"{BASE_OSS_INSIGHT_URL}/collections"
PERIOD_VALUES = ["past_24_hours", "past_week", "past_month", "past_3_months"]

# Initialize API Clients
openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_PROJECT_KEY"))
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))
github_client = Github(os.getenv("GITHUB_TOKEN"))

# LLM Model Configuration
REASONING_MODELS = [
    "o1", "o1-2024-12-17", "o3-mini-2025-01-31", "o3-mini",
    "o4-mini", "o4-mini-2025-04-16", "o3", "o3-2025-04-16"
]

LLM_MODEL_COSTS = {
    "o3": {"input": 2 / 10 ** 6, "cached_input": 0.6 / 10 ** 6, "output": 8 / 10 ** 6},
    "o3-2025-04-16": {"input": 10.0 / 10 ** 6, "cached_input": 2.5 / 10 ** 6, "output": 40.0 / 10 ** 6},
    "o4-mini": {"input": 1.1 / 10 ** 6, "cached_input": 0.275 / 10 ** 6, "output": 4.4 / 10 ** 6},
    "o4-mini-2025-04-16": {"input": 1.1 / 10 ** 6, "cached_input": 0.275 / 10 ** 6, "output": 4.4 / 10 ** 6},
    "gpt-4.1-mini": {"input": 0.40 / 10 ** 6, "cached_input": 0.10 / 10 ** 6, "output": 1.60 / 10 ** 6},
    "gpt-4.1": {"input": 2.00 / 10 ** 6, "cached_input": 0.50 / 10 ** 6, "output": 8.00 / 10 ** 6},
    "gpt-4.1-nano": {"input": 0.10 / 10 ** 6, "cached_input": 0.025 / 10 ** 6, "output": 0.40 / 10 ** 6},
    "gpt-4o": {"input": 2.5 / 10 ** 6, "cached_input": 1.25 / 10 ** 6, "output": 10 / 10 ** 6},
    "gpt-4o-2024-11-20": {"input": 2.5 / 10 ** 6, "cached_input": 1.25 / 10 ** 6, "output": 10 / 10 ** 6},
    "gpt-4o-2024-08-06": {"input": 2.5 / 10 ** 6, "cached_input": 1.25 / 10 ** 6, "output": 10 / 10 ** 6},
    "gpt-4o-mini": {"input": 0.15 / 10 ** 6, "cached_input": 0.075 / 10 ** 6, "output": 0.6 / 10 ** 6},
    "text-embedding-3-small": {"input": 0.02 / 10 ** 6},
    "text-embedding-3-large": {"input": 0.13 / 10 ** 6},
    "text-embedding-ada-002": {"input": 0.1 / 10 ** 6}
}

# Pydantic Models for VC Analysis
class Collection(BaseModel):
    id: str
    name: str

class DevOpsCollectionsResponse(BaseModel):
    collections: List[Collection]

class DevOpsClassification(BaseModel):
    is_devops_related: bool  # True if this repository is DevOps/infrastructure related
    confidence_score: int  # Score from 1-5, how confident the classification is
    reasoning: str  # Brief explanation of why it's classified as DevOps or not
    devops_categories: List[str]  # Categories like "CI/CD", "Container", "Monitoring", etc.

class BigTechClassification(BaseModel):
    is_big_tech: bool = Field(
        ...,
        description="True if this repository appears to be from a large/established company unsuitable for early-stage VC investment"
    )
    confidence_score: int = Field(
        ...,
        description="Confidence level (1-5, where 5 is very confident)",
        ge=1, le=5
    )
    reasoning: str = Field(
        ...,
        description="Brief explanation of the classification decision"
    )
    company_indicators: List[str] = Field(
        ...,
        description="List of indicators found (e.g., 'Enterprise features', 'Large team size', 'Corporate backing')"
    )

class RepoRubricAnalysis(BaseModel):
    """A Pydantic model for the structured output of the README rubric analysis."""
    problem_clarity_score: int = Field(
        ...,
        description="Score (1-5) for how clearly the project states the problem it solves.",
        ge=1,
        le=5
    )
    adoption_ease_score: int = Field(
        ...,
        description="Score (1-5) for how easy it is to get started based on the README.",
        ge=1,
        le=5
    )
    maturity_health_score: int = Field(
        ...,
        description="Score (1-5) for project maturity as inferred from README links and content.",
        ge=1,
        le=5
    )
    problem_solved: str = Field(
        ...,
        description="In one sentence, what specific, frustrating problem does this project solve?"
    )

class CommunitySentimentAnalysis(BaseModel):
    """Structured output for community sentiment analysis."""
    excitement_score: int = Field(
        ...,
        description="Score (1-5) for the community's emotional energy and hype.",
        ge=1, le=5
    )
    problem_solution_fit_score: int = Field(
        ...,
        description="Score (1-5) for how well the project's value proposition resonates with the community.",
        ge=1, le=5
    )
    credibility_adoption_score: int = Field(
        ...,
        description="Score (1-5) based on the authority of commenters and signs of real-world adoption.",
        ge=1, le=5
    )
    key_praise_quote: str = Field(
        ...,
        description="A single, direct quote that best captures the community's core praise."
    )
    main_criticism: str = Field(
        ...,
        description="A one-sentence summary of the most common or significant criticism."
    )

class FinalAnalysis(BaseModel):
    repo_name: str
    oss_insight_data: dict
    repo_rubric_analysis: Optional[RepoRubricAnalysis] = None
    community_sentiment_analysis: Optional[CommunitySentimentAnalysis] = None
    score: Optional[float] = None  # Calculated final score

print("✅ VC Sourcing Pipeline initialized successfully")
print(f"🔑 API clients configured for OpenAI, Exa, and GitHub")
print(f"📊 Ready to analyze DevOps repositories across {len(PERIOD_VALUES)} time periods")

✅ VC Sourcing Pipeline initialized successfully
🔑 API clients configured for OpenAI, Exa, and GitHub
📊 Ready to analyze DevOps repositories across 4 time periods


In [126]:
# Cost Tracking for LLM Usage
class ModelUsageAsync:
    """Track LLM API costs throughout the VC analysis pipeline."""
    
    def __init__(self, model_costs=LLM_MODEL_COSTS, web_search_costs=None):
        self.token_usage = {}
        self.web_search_usage = {}
        self.model_costs = model_costs
        self.web_search_costs = web_search_costs
        self.logs = []
        self.lock = asyncio.Lock()

    async def add_tokens(
        self, 
        model: str, 
        input_tokens: int = 0, 
        output_tokens: int = 0, 
        cached_tokens: int = 0, 
        reasoning_tokens: int = 0, 
        label: Optional[str] = None
    ) -> None:
        """Add token usage for cost tracking."""
        async with self.lock:
            if model not in self.token_usage:
                self.token_usage[model] = {"input": 0, "output": 0, "cached": 0, "reasoning": 0}
            
            self.token_usage[model]["input"] += input_tokens
            self.token_usage[model]["output"] += output_tokens
            self.token_usage[model]["cached"] += cached_tokens
            self.token_usage[model]["reasoning"] += reasoning_tokens
            
            if label is not None:
                self.logs.append({
                    "model": model, "input_tokens": input_tokens, "output_tokens": output_tokens,
                    "cached_tokens": cached_tokens, "reasoning_tokens": reasoning_tokens, "label": label
                })

    async def add_web_search_usage(self, model: str, search_context_size: str) -> None:
        """Add web search usage for cost tracking."""
        if self.web_search_costs is not None:
            if model not in self.web_search_usage:
                self.web_search_usage[model] = {}
            if search_context_size not in self.web_search_usage[model]:
                self.web_search_usage[model][search_context_size] = 0
            self.web_search_usage[model][search_context_size] += 1
        
    async def get_cost(self) -> float:
        """Calculate total cost of LLM usage."""
        async with self.lock:
            for model in self.token_usage:
                if model not in self.model_costs:
                    raise ValueError(f"Model {model} is not supported.")
            
            cost_of_input_output = sum([
                (self.token_usage[model]["input"] - self.token_usage[model]["cached"]) * self.model_costs[model].get("input", 0) +
                self.token_usage[model]["output"] * self.model_costs[model].get("output", 0) +
                self.token_usage[model]["cached"] * self.model_costs[model].get("cached_input", 0)
                for model in self.token_usage 
            ]) 

            cost_of_web_searches = 0
            if self.web_search_costs is not None:
                for model in self.web_search_usage:
                    for search_context_size in self.web_search_usage[model]:
                        cost_of_web_searches += self.web_search_usage[model][search_context_size] * self.web_search_costs[model].get(search_context_size, 0)

            return cost_of_input_output + cost_of_web_searches

    async def get_tokens_used(self) -> int:
        """Get total number of tokens used."""
        async with self.lock:
            return sum([
                self.token_usage[model]["input"] + self.token_usage[model]["output"]
                for model in self.token_usage
            ])

    async def get_web_searches_performed(self) -> int:
        """Get total number of web searches performed."""
        async with self.lock:
            total_searches = 0
            for model in self.web_search_usage:
                for search_context_size in self.web_search_usage[model]:
                    total_searches += self.web_search_usage[model][search_context_size]
            return total_searches


In [127]:
# OpenAI API Helper with Structured Output
async def call_openai_structured(
    openai_client: AsyncOpenAI,
    model: str,
    llm_usage: Optional[ModelUsageAsync] = None,
    llm_usage_label: Optional[str] = None,
    max_retries: int = 3,
    base_delay: int = 9,
    completion_timeout: int = 120,
    web_search_bool: bool = False,
    web_search_context_size: str = "high",
    **kwargs
) -> Optional[ChatCompletion]:
    """Call OpenAI API with structured output and cost tracking."""
    
    # Handle model-specific parameters
    if model in REASONING_MODELS:
        kwargs.pop("temperature", None)  # Reasoning models don't support temperature
    else:
        kwargs.pop("reasoning_effort", None)  # Only supported by reasoning models
    
    for attempt in range(max_retries):
        try:
            completion = await asyncio.wait_for(
                openai_client.chat.completions.parse(model=model, **kwargs),
                timeout=completion_timeout
            )
            
            # Track usage and costs
            if llm_usage is not None:
                await llm_usage.add_tokens(
                    model=model,
                    input_tokens=completion.usage.prompt_tokens,
                    output_tokens=completion.usage.completion_tokens,
                    cached_tokens=completion.usage.prompt_tokens_details.cached_tokens,
                    reasoning_tokens=completion.usage.completion_tokens_details.reasoning_tokens,
                    label=llm_usage_label
                )
                
                if web_search_bool:
                    await llm_usage.add_web_search_usage(model, web_search_context_size)
            
            return completion
            
        except asyncio.TimeoutError:
            print(f"🔄 API timeout for {llm_usage_label or 'request'}, retrying...")
            await asyncio.sleep(base_delay)
        except Exception as e:
            print(f"❌ API error for {llm_usage_label or 'request'}: {e}")
            if attempt == max_retries - 1:
                break
            await asyncio.sleep(base_delay)

    print(f"❌ Max retries exceeded for {llm_usage_label or 'request'}")
    return None


In [128]:
# OSS Insight API Helper
async def make_oss_insight_request(
    session: aiohttp.ClientSession,
    url: str,
    params: dict
) -> Optional[dict]:
    """Make requests to OSS Insight API with error handling."""
    try:
        async with session.get(url, params=params) as response:
            response.raise_for_status()
            data = await response.json()
            return data
    except Exception as e:
        print(f"❌ OSS Insight API error: {e}")
        return None

In [129]:
# DevOps Collections Identification
GET_DEV_OPS_COLLECTIONS_SYSTEM_PROMPT = """
You are a helpful assistant that identifies DevOps-related collections from a list.
Use a broad, inclusive definition of DevOps: include any tools that developers might use for infrastructure, CI/CD, deployment, monitoring, automation, observability, or general backend workflows.
It's better to include a collection that might not be strictly DevOps than to exclude something useful to developers.
"""

GET_DEV_OPS_COLLECTIONS_USER_PROMPT = """
Task: Identify DevOps-related collections from the list below.
Use a broad definition of DevOps. Include tools for:
- infrastructure, CI/CD, deployment, monitoring, observability, automation
- cloud services, containerization, developer productivity (especially backend)
- workflow tooling, build systems, scripting, package management

Exclude only obviously non-relevant tools like:
- frontend-only UI libraries (e.g., Tailwind, React)
- general note-taking or static content tools

List of collections:
{context}

Your answer must:
1. Only contain collections from the provided list
2. Return collections in the specified format with both id and name fields
3. Be inclusive of any tools that developers would use for infrastructure, automation, or deployment
"""

async def get_dev_ops_collections(
    openai_client: AsyncOpenAI,
    collections_data: dict,
    llm_usage: ModelUsageAsync
) -> Optional[DevOpsCollectionsResponse]:
    """Identify DevOps collections using LLM analysis."""
    collections_rows = collections_data.get("data", {}).get("rows", [])
    all_collections = [{row["id"]: row["name"]} for row in collections_rows]
    
    print(f"🔍 Analyzing {len(all_collections)} collections to identify DevOps-related ones...")
    
    context = "\n".join(f"{row['id']}: {row['name']}" for row in collections_rows)
    messages = [
        {"role": "developer", "content": GET_DEV_OPS_COLLECTIONS_SYSTEM_PROMPT},
        {"role": "user", "content": GET_DEV_OPS_COLLECTIONS_USER_PROMPT.format(context=context)}
    ]

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o3",
        messages=messages,
        response_format=DevOpsCollectionsResponse,
        llm_usage=llm_usage,
        llm_usage_label="dev_ops_collections_identification",
        reasoning_effort="high"
    )

    if not response or not response.choices[0].message.parsed:
        print("❌ Failed to identify DevOps collections")
        return None
    
    parsed_response = response.choices[0].message.parsed

    print(f"✅ Identified {len(parsed_response.collections)} DevOps collections")
    return parsed_response.collections

# Testing get_dev_ops_collections
# print("Testing get_dev_ops_collections")
# async with aiohttp.ClientSession() as session:
#     collections_data = await make_oss_insight_request(
#         session=session,
#         url=COLLECTIONS_URL,
#         params={}
#     )

# if collections_data:
#     test_usage = ModelUsageAsync()
#     dev_ops_collections = await get_dev_ops_collections(openai_client, collections_data, test_usage)
#     if dev_ops_collections:
#         dev_ops_collections = [{c.id: c.name} for c in dev_ops_collections]
#         print(dev_ops_collections)

#     # Display cost information
#     total_cost = await test_usage.get_cost()
#     total_tokens = await test_usage.get_tokens_used()
#     print(f"\n💸 Test Analysis Cost: ${total_cost:.4f}")
#     print(f"🔢 Test Total Tokens Used: {total_tokens:,}")


In [130]:
# LLM-Based DevOps Repository Classification
CLASSIFY_DEVOPS_REPO_SYSTEM_PROMPT = """
You are an expert VC analyst specializing in DevOps and infrastructure tooling. Your task is to classify whether a GitHub repository is related to DevOps, infrastructure, or developer productivity tools.
Be loose with the definition of DevOps.
It's better to include a repository that might not be strictly DevOps than to exclude something useful to developers.
Also include markup frameworks, documentation tools, and other tools that developers could use in their daily work.
"""

CLASSIFY_DEVOPS_REPO_USER_PROMPT = """
Repository: {repo_name}
Description: {description}

Based on the repository name and description, determine if this is a DevOps-related project.

DevOps categories include:
- Any tools that developers could use in their daily work; be loose with the definition of DevOps
- CI/CD and deployment pipelines
- Container technologies (Docker, Kubernetes, etc.)
- Infrastructure as Code (Terraform, Ansible, etc.)
- Monitoring and observability tools
- Cloud platforms and services
- Developer productivity tools
- Security and compliance tools
- Database and data infrastructure
- API gateways and service mesh
- Configuration management
- Build and package management tools
- Tools for building or deploying AI/ML models

Analyze this repository and provide:
1. **is_devops_related**: True if it's DevOps/infrastructure related
2. **confidence_score**: Your confidence (1-5, where 5 is very confident)
3. **reasoning**: Brief explanation of your classification
4. **devops_categories**: List of relevant DevOps categories (empty list if not DevOps)
"""

async def classify_repository_with_llm(
    openai_client: AsyncOpenAI,
    repo_name: str,
    description: str,
    llm_usage: ModelUsageAsync
) -> Optional[DevOpsClassification]:
    """Classify a repository as DevOps-related using LLM analysis."""
    
    messages = [
        {"role": "developer", "content": CLASSIFY_DEVOPS_REPO_SYSTEM_PROMPT},
        {"role": "user", "content": CLASSIFY_DEVOPS_REPO_USER_PROMPT.format(
            repo_name=repo_name,
            description=description or "No description available"
        )}
    ]

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=messages,
        response_format=DevOpsClassification,
        llm_usage=llm_usage,
        llm_usage_label=f"devops_classification_{repo_name}",
        reasoning_effort="medium"
    )

    if response and response.choices:
        return response.choices[0].message.parsed
    
    return None

print("✅ LLM-based DevOps classification function loaded")

# Test classify_repository_with_llm
# print("Testing classify_repository_with_llm")
# test_repos = [{"name": "LibreSpark/LibreTV", "description": "一分钟搭建影视站，支持Vercel/Docker等部署方式."}, {"name": "srbhr/Resume-Matcher", "description": "Improve your resumes with Resume Matcher. Get insights, keyword suggestions and tune your resumes to job descriptions."}, {"name": "maybe-finance/maybe", "description": "The personal finance app for everyone"}, {"name": "roboflow/supervision", "description": "We write your reusable computer vision tools. 💜"}, {"name": "bluesky-social/social-app", "description": "The Bluesky Social application for Web, iOS, and Android"}, {"name": "typst/typst", "description": "A new markup-based typesetting system that is powerful and easy to learn."}, {"name": "aidenybai/react-scan", "description": "Scan for React performance issues and eliminate slow renders in your app"}]
# test_usage = ModelUsageAsync()
# for repo in test_repos:
#     print(repo["name"])
#     print(repo["description"])
#     print(await classify_repository_with_llm(openai_client, repo["name"], repo["description"], test_usage))
#     print("-"*100)

# # Display cost information
# total_cost = await test_usage.get_cost()
# total_tokens = await test_usage.get_tokens_used()
# print(f"\n💸 Test Analysis Cost: ${total_cost:.4f}")
# print(f"🔢 Test Total Tokens Used: {total_tokens:,}")

✅ LLM-based DevOps classification function loaded


In [131]:
# Helper Functions for VC Analysis
def truncate_content_for_llm(content: str, max_chars: int = 15000) -> str:
    """Truncate content to fit within LLM context limits."""
    return content[:max_chars] if len(content) > max_chars else content

In [132]:
# Big Tech Company Classification
BIG_TECH_CLASSIFICATION_SYSTEM_PROMPT = """
You are a VC analyst specializing in identifying early-stage investment opportunities. Your task is to determine if a GitHub repository belongs to a large, established company that would be unsuitable for early-stage VC investment.

Focus on indicators that suggest the company is too mature/large for early-stage funding.
"""

BIG_TECH_CLASSIFICATION_USER_PROMPT = """
Repository: {repo_name}
Description: {description}

README Content:
---
{readme_content}
---

Analyze this repository to determine if it belongs to a large, established company unsuitable for early-stage VC investment.

**EXCLUDE (mark as big tech) if you find:**
- Clear corporate backing (Google, Meta, Microsoft, Amazon, Apple, Anthropic, OpenAI, etc.)
- Enterprise-focused language ("Enterprise", "at scale", "production-grade for large organizations")
- Large team indicators (many contributors, corporate structure mentions)
- Established company features (SOC2 compliance, enterprise sales, dedicated support teams)
- Corporate open-source projects from known large companies
- References to IPO, public trading, or being part of Fortune 500 companies

**INCLUDE (mark as NOT big tech) if:**
- Startup/indie language ("we're building", "early stage", "small team")
- Personal projects or small team indicators
- Seeking funding, investors, or early adopters
- Grassroots community-driven projects
- Clear startup indicators (YC batch, seed funding, small team size)

Provide:
1. **is_big_tech**: True if unsuitable for early-stage VC (big tech/established company)
2. **confidence_score**: Your confidence (1-5)
3. **reasoning**: Brief explanation of your decision
4. **company_indicators**: List of specific indicators found
"""

async def classify_big_tech_company(
    openai_client: AsyncOpenAI,
    github_client: Github,
    repo_name: str,
    description: str,
    llm_usage: ModelUsageAsync
) -> Optional[BigTechClassification]:
    """Classify if repository belongs to big tech company unsuitable for VC investment."""
    
    # Fetch README content
    try:
        repo = await asyncio.to_thread(github_client.get_repo, repo_name)
        readme_obj = await asyncio.to_thread(repo.get_readme)
        readme_content = readme_obj.decoded_content.decode("utf-8")
        readme_content = truncate_content_for_llm(readme_content, max_chars=10000)  # Smaller limit for speed
        print(f"✅ Fetched README for big tech classification: {repo_name}")
    except Exception as e:
        print(f"⚠️ Could not fetch README for {repo_name}: {e}")
        readme_content = "README not available"
    
    messages = [
        {"role": "developer", "content": BIG_TECH_CLASSIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": BIG_TECH_CLASSIFICATION_USER_PROMPT.format(
            repo_name=repo_name,
            description=description or "No description available",
            readme_content=readme_content
        )}
    ]

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",  # Fast model for this check
        messages=messages,
        response_format=BigTechClassification,
        llm_usage=llm_usage,
        llm_usage_label=f"big_tech_classification_{repo_name}",
        reasoning_effort="low"  # Fast classification
    )

    if response and response.choices:
        return response.choices[0].message.parsed
    
    return None

print("✅ Big tech classification function loaded")

# Test classify_big_tech_company
# print("Testing classify_big_tech_company")
# test_repos = [{"name": "anthropics/claude-code", "description": "Claude Code is an agentic coding tool that lives in your terminal, understands your codebase, and helps you code faster by executing routine tasks, explaining complex code, and handling git workflows - all through natural language commands."}, {"name": "google-gemini/gemini-cli", "description": "An open-source AI agent that brings the power of Gemini directly into your terminal."}, {"name": "maybe-finance/maybe", "description": "The personal finance app for everyone"}, {"name": "roboflow/supervision", "description": "We write your reusable computer vision tools. 💜"}, {"name": "bluesky-social/social-app", "description": "The Bluesky Social application for Web, iOS, and Android"}, {"name": "typst/typst", "description": "A new markup-based typesetting system that is powerful and easy to learn."}, {"name": "aidenybai/react-scan", "description": "Scan for React performance issues and eliminate slow renders in your app"}, {"name": "n8n-io/n8n", "description": "Fair-code workflow automation platform with native AI capabilities. Combine visual building with custom code, self-host or cloud, 400+ integrations."}]
# test_usage = ModelUsageAsync()
# for repo in test_repos:
#     print(repo["name"])
#     print(repo["description"])
#     print(await classify_big_tech_company(openai_client, github_client, repo["name"], repo["description"], test_usage))
#     print("-"*100)

# # Display cost information
# total_cost = await test_usage.get_cost()
# total_tokens = await test_usage.get_tokens_used()
# print(f"\n💸 Test Analysis Cost: ${total_cost:.4f}")
# print(f"🔢 Test Total Tokens Used: {total_tokens:,}")


✅ Big tech classification function loaded


In [133]:
# Getting files for analysis from GitHub
async def _get_file_details(
    repo: Repository,
    file_basename: str,
) -> Optional[Dict]:
    """
    Fetches a specific file's content and its last commit date from a GitHub repo.

    Args:
        repo: The repository object.
        file_basename: The basename of the file (e.g., 'CONTRIBUTING').

    Returns:
        A dictionary containing the file's content and last modification date,
        or None if the file doesn't exist.
    """
    file_path_variants = [
        f"{file_basename.upper()}.md",
        f"{file_basename.lower()}.md",
        f".github/{file_basename.upper()}.md",
        f".github/{file_basename.lower()}.md",
    ]
    for path in file_path_variants:
        try:
            print(f"🔎 Checking for '{path}' in {repo.name}...")
            content_file = await asyncio.to_thread(repo.get_contents, path)
            files_content = content_file.decoded_content.decode("utf-8")

            commits = await asyncio.to_thread(repo.get_commits, path=path)
            last_commit_date = commits[0].commit.committer.date

            print(f"✅ Found '{path}' in {repo.name}")
            return {
                "content": files_content,
                "last_modified": last_commit_date
            }
        except UnknownObjectException:
            # This exception is raised when the file is not found (404)
            continue
        except Exception as e:
            print(f"❌ Could not fetch '{path}' from {repo.name}: {e}")
            return None

    print(f"❌ Could not fetch '{file_basename}' from {repo.name}")
    return None
        
# Repository Analysis Function
async def analyze_repo_with_rubric(
    openai_client: AsyncOpenAI,
    github_client: Github,
    repo_name: str,
    llm_usage: ModelUsageAsync
) -> Optional[RepoRubricAnalysis]:
    """Analyze repository README and other files for VC investment potential."""
    try:
        # Get the repository
        repo = await asyncio.to_thread(github_client.get_repo, repo_name)

        # Get the README file details
        readme_obj = await asyncio.to_thread(repo.get_readme)
        readme_content = readme_obj.decoded_content.decode("utf-8")
        readme_content = truncate_content_for_llm(readme_content)
        print(f"✅ Fetched README for {repo_name}")
    except Exception as e:
        print(f"❌ Could not fetch README for {repo_name}: {e}")
        return None

    # Get the file details for CONTRIBUTING.md
    contributing_details = await _get_file_details(
        repo=repo,
        file_basename="CONTRIBUTING"
    )
    contributing_content = None
    contributing_last_modified = None
    if contributing_details:
        contributing_content = contributing_details["content"]
        contributing_content = truncate_content_for_llm(
            content=contributing_content, 
            max_chars=2000
        )
        contributing_last_modified = contributing_details["last_modified"]

    # Get the file details for CODE_OF_CONDUCT.md
    code_of_conduct_details = await _get_file_details(
        repo=repo,
        file_basename="CODE_OF_CONDUCT"
    )
    code_of_conduct_content = None
    code_of_conduct_last_modified = None
    if code_of_conduct_details:
        code_of_conduct_content = code_of_conduct_details["content"]
        code_of_conduct_content = truncate_content_for_llm(
            content=code_of_conduct_content, 
            max_chars=2000
        )
        code_of_conduct_last_modified = code_of_conduct_details["last_modified"]

    system_prompt = "You are a sharp and discerning VC analyst specializing in DevOps and open-source software. Your task is to apply a specific rubric to evaluate a project's investment potential based on its GitHub presence."

    user_prompt = f"""
    Analyze the following repository: '{repo_name}'.

    **Repository Data:**
    - Found 'CONTRIBUTING.md': {contributing_content is not None}
    - Found 'CODE_OF_CONDUCT.md': {code_of_conduct_content is not None}
    - Last modified 'CONTRIBUTING.md': {contributing_last_modified.strftime('%Y-%m-%d') if contributing_last_modified else 'N/A'}
    - Last modified 'CODE_OF_CONDUCT.md': {code_of_conduct_last_modified.strftime('%Y-%m-%d') if code_of_conduct_last_modified else 'N/A'}

    **README Content:**
    ---
    {readme_content}
    ---

    **Contributing Content:**
    ---
    {contributing_content or "No contributing file found"}
    ---

    **Code of Conduct Content:**
    ---
    {code_of_conduct_content or "No code of conduct file found"}
    ---

    **Your Task:**
    Evaluate the project strictly based on the rubric below and provide a score from 1 to 5 for each category. Do not provide explanations, only the scores.

    **Rubric for Evaluation:**

    **1. Problem Statement Clarity & Value Proposition (Score 1-5):**
    - 1 (Poor): Unclear what problem it solves.
    - 3 (Average): Describes *what* it does, but not *why* it's a compelling solution.
    - 5 (Excellent): Clearly explains a frustrating 'painkiller' problem and presents itself as the obvious solution.

    **2. Ease of Adoption / 'Time to First Wow' (Score 1-5):**
    - 1 (Poor): No clear 'Getting Started' guide or examples.
    - 3 (Average): Basic installation instructions are present but might be complex.
    - 5 (Excellent): A clear 'Quickstart' section with copy-paste commands to get a user to a 'wow' moment in minutes.

    **3. Project Maturity & Community Health (Score 1-5):**
    - 1 (Poor): No `CONTRIBUTING.md` or links to a community. Last commit was over 3 months ago.
    - 3 (Average): Has either a `CONTRIBUTING.md` or a community link. Last commit was within the last month.
    - 5 (Excellent): Has a `CONTRIBUTING.md`, a `CODE_OF_CONDUCT.md`, and links to an active community. Last commit was within the last week.

    Required Output:**

    1.  **Problem Statement Clarity (Score 1-5):**
    2.  **Ease of Adoption (Score 1-5):** 
    3.  **Project Maturity & Community Health (Score 1-5):
    4.  **Problem Solved (One sentence):**
        -   In a single sentence, describe the specific, frustrating problem this project solves.
    """
    
    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=[{"role": "developer", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        response_format=RepoRubricAnalysis,
        llm_usage=llm_usage,
        llm_usage_label=f"readme_analysis_{repo_name}",
        reasoning_effort="high"
    )

    if response and response.choices:
        print(f"✅ README analysis completed for {repo_name}")
        return response.choices[0].message.parsed
    
    print(f"❌ README analysis failed for {repo_name}")
    return None

# Test analyze_repo_with_rubric
# print("Testing analyze_repo_with_rubric")
# test_usage = ModelUsageAsync()
# test_repos = ["langchain-ai/langchain", "supabase/supabase", "typst/typst", "aidenybai/million", "aidenybai/react-scan"]
# for repo in test_repos:
#     print(f"Testing {repo}")
#     print(await analyze_repo_with_rubric(openai_client, github_client, repo, test_usage))
#     print("-"*100)

# # Display cost information
# total_cost = await test_usage.get_cost()
# total_tokens = await test_usage.get_tokens_used()
# print(f"\n💸 Test Analysis Cost: ${total_cost:.4f}")
# print(f"🔢 Test Total Tokens Used: {total_tokens:,}")

In [134]:
# Community Sentiment Analysis Function
async def analyze_community_sentiment(
    openai_client: AsyncOpenAI,
    exa_client: Exa,
    repo_name: str,
    llm_usage: ModelUsageAsync
) -> Optional[CommunitySentimentAnalysis]:
    """Analyze community sentiment using Exa search and LLM analysis."""
    # Use Exa to find relevant discussions
    query = f"discussions and reviews of the GitHub repository '{repo_name}'"
    try:
        search_response = exa_client.search_and_contents(
            query,
            num_results=5,
            include_domains=["news.ycombinator.com", "reddit.com"],
            text=True
        )
        
        if not search_response.results:
            print(f"⚠️ No Exa search results found for {repo_name}")
            return None
            
        # Format the context for the LLM
        context = "\n\n".join([
            f"URL: {res.url}\nContent: {res.text}" 
            for res in search_response.results
        ])
        print(f"✅ Found {len(search_response.results)} community discussions for {repo_name}")
        
    except Exception as e:
        print(f"❌ Exa search failed for {repo_name}: {e}")
        return None

    system_prompt = "You are a VC analyst specializing in developer sentiment. Your task is to analyze discussions from Hacker News and Reddit to evaluate a project's community perception using a specific rubric."

    user_prompt = f"""
    Project: {repo_name}

    Discussion Threads:
    ---
    {context}
    ---

    **Your Task:**
    Analyze the discussion threads and provide a structured analysis based on the rubric below.

    **Rubric for Evaluation:**

    **1. Community Excitement Score (1-5):**
    - **1 (Dismissive):** Predominantly negative. Commenters are critical or uninterested.
    - **3 (Curious):** Neutral to positive. Comments like "neat idea" or "I'll keep an eye on this." No strong emotion.
    - **5 (Evangelical):** Viral excitement. Comments like "game-changer," "I'm switching to this immediately," or "finally, someone solved this."

    **2. Problem-Solution Fit Score (1-5):**
    - **1 (Mismatch):** Community doesn't believe the problem is real or thinks the solution is wrong. Many "Why not just use X?" comments.
    - **3 (Acknowledged):** The community agrees the problem is real but may be unconvinced this is the best solution.
    - **5 (Strong Resonance):** The community immediately validates the pain point. Comments like "I desperately need this," or "This solves a huge headache for my team."

    **3. Credibility & Adoption Score (1-5):**
    - **1 (Low):** Praise is superficial, or there are significant, unaddressed technical critiques. No one mentions using it.
    - **3 (Moderate):** Positive discussion, but mainly from anonymous users. A few people mention they are "trying it out."
    - **5 (High):** Praise comes from users with stated authority (e.g., "Principal Engineer at X"). Multiple commenters report successful adoption in personal or professional projects.

    **Required Output:**
    Based on your analysis, provide the five required fields: `excitement_score`, `problem_solution_fit_score`, `credibility_adoption_score`, `key_praise_quote`, and `main_criticism`.
    """

    response = await call_openai_structured(
        openai_client=openai_client,
        model="o4-mini",
        messages=[{"role": "developer", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        response_format=CommunitySentimentAnalysis,
        llm_usage=llm_usage,
        llm_usage_label=f"sentiment_analysis_{repo_name}",
        reasoning_effort="high"
    )
    
    if response and response.choices:
        print(f"✅ Sentiment analysis completed for {repo_name}")
        return response.choices[0].message.parsed
    
    print(f"❌ Sentiment analysis failed for {repo_name}")
    return None

    print(user_prompt)

# Test analyze_community_sentiment
# print("Testing analyze_community_sentiment")
# test_usage = ModelUsageAsync()
# test_repos = ["langchain-ai/langchain", "supabase/supabase", "typst/typst", "aidenybai/million", "aidenybai/react-scan"]
# for repo in test_repos:
#     print(f"Testing {repo}")
#     print(await analyze_community_sentiment(openai_client, exa_client, repo, test_usage))
#     print("-"*100)

# # Display cost information
# total_cost = await test_usage.get_cost()
# total_tokens = await test_usage.get_tokens_used()
# print(f"\n💸 Test Analysis Cost: ${total_cost:.4f}")
# print(f"🔢 Test Total Tokens Used: {total_tokens:,}")


In [135]:
def calculate_final_score(analysis: 'FinalAnalysis') -> float:
    """
    Calculate a final, weighted investment score based on multiple factors.
    
    This version uses explicit weights and normalizes each component to a 0-10 scale
    before applying weights, making the system easier to tune.
    """
    
    # --- Step 1: Define Weights ---
    # These can be easily tuned to adjust the ranking algorithm.
    weights = {
        "popularity": 1.0,
        "repo_quality": 1.5,
        "community_sentiment": 1.8  # Weighs sentiment ~20% more than repo quality
    }
    
    # --- Step 2: Calculate Individual Component Scores (Normalized to 0-10) ---
    
    # 1. Popularity Score (Log-normalized)
    # A score of 22000 (like supabase) gives ~9.0. A score of 100 gives ~4.6.
    # This captures order-of-magnitude popularity better.
    total_score_raw = float(analysis.oss_insight_data.get("total_score", 0))
    # Add 1 to avoid math.log(0). The base of the log can be tuned.
    popularity_score = math.log(total_score_raw + 1, 10) * 2.22 if total_score_raw > 0 else 0.0
    popularity_score = min(popularity_score, 10.0) # Cap at 10

    # 2. Repository Rubric Quality Score
    repo_score = 0.0
    if analysis.repo_rubric_analysis:
        repo_sum = (
            analysis.repo_rubric_analysis.problem_clarity_score +
            analysis.repo_rubric_analysis.adoption_ease_score +
            analysis.repo_rubric_analysis.maturity_health_score
        )
        # Normalize from a 3-15 scale to a 0-10 scale
        repo_score = ((repo_sum - 3) / 12) * 10

    # 3. Community Sentiment Score
    sentiment_score = 0.0
    if analysis.community_sentiment_analysis:
        sentiment_sum = (
            analysis.community_sentiment_analysis.excitement_score +
            analysis.community_sentiment_analysis.problem_solution_fit_score +
            analysis.community_sentiment_analysis.credibility_adoption_score
        )
        # Normalize from a 3-15 scale to a 0-10 scale
        sentiment_score = ((sentiment_sum - 3) / 12) * 10
        
    # --- Step 3: Apply Weights and Calculate Final Score ---
    
    final_score = (
        (popularity_score * weights["popularity"]) +
        (repo_score * weights["repo_quality"]) +
        (sentiment_score * weights["community_sentiment"])
    )
    
    # Optional: Normalize the final score to a 0-100 scale for easier reading
    total_weight = sum(weights.values())
    final_score_normalized = (final_score / (10 * total_weight)) * 100
    
    return round(final_score_normalized, 2)

# Repository Analysis Pipeline
async def analyze_repository_pipeline(
    repo_name: str,
    repo_data: dict,
    openai_client: AsyncOpenAI,
    github_client: Github,
    exa_client: Exa,
    llm_usage: ModelUsageAsync
) -> FinalAnalysis:
    """Complete analysis pipeline for a single repository."""
    print(f"🔍 Analyzing {repo_name}...")
    
    # Parallel analysis where possible
    repo_rubric_task = analyze_repo_with_rubric(openai_client, github_client, repo_name, llm_usage)
    sentiment_task = analyze_community_sentiment(openai_client, exa_client, repo_name, llm_usage)
    
    repo_rubric_analysis, community_sentiment_analysis = await asyncio.gather(
        repo_rubric_task,
        sentiment_task,
        return_exceptions=True
    )
    
    # Handle exceptions
    if isinstance(repo_rubric_analysis, Exception):
        print(f"⚠️ README analysis failed for {repo_name}: {repo_rubric_analysis}")
        repo_rubric_analysis = None
    
    if isinstance(community_sentiment_analysis, Exception):
        print(f"⚠️ Sentiment analysis failed for {repo_name}: {community_sentiment_analysis}")
        community_sentiment_analysis = None
    
    # Create final analysis
    analysis = FinalAnalysis(
        repo_name=repo_name,
        oss_insight_data=repo_data,
        repo_rubric_analysis=repo_rubric_analysis,
        community_sentiment_analysis=community_sentiment_analysis
    )
    
    # Calculate score
    analysis.score = calculate_final_score(analysis)
    
    return analysis


In [136]:
# Reporting Functions
def print_analysis_summary(analysis: FinalAnalysis) -> None:
    """Print formatted analysis summary."""
    print(f"\n## {analysis.repo_name} | 📊 Score: {analysis.score}")
    print(f"📈 Total Score: {analysis.oss_insight_data.get('total_score')} | ⭐ Stars: {analysis.oss_insight_data.get('stars')}")
    print(f"📝 Description: {analysis.oss_insight_data.get('description')}")
    
    if analysis.repo_rubric_analysis:
        print(f"📖 Problem Solved: {analysis.repo_rubric_analysis.problem_solved}")
        print(f"🎯 Problem Statement Clarity: {analysis.repo_rubric_analysis.problem_clarity_score}/5")
        print(f"⚡ Ease of Adoption: {analysis.repo_rubric_analysis.adoption_ease_score}/5")
        print(f"💊 Project Maturity & Community Health: {analysis.repo_rubric_analysis.maturity_health_score}/5")
    
    if analysis.community_sentiment_analysis:
        print(f"😊 Community Sentiment: {analysis.community_sentiment_analysis.excitement_score}/5")
        print(f"📈 Problem Solution Fit: {analysis.community_sentiment_analysis.problem_solution_fit_score}/5")
        print(f"💊 Credibility & Adoption: {analysis.community_sentiment_analysis.credibility_adoption_score}/5")
        if analysis.community_sentiment_analysis.key_praise_quote:
            print(f"💬 Key Praise Quote: {analysis.community_sentiment_analysis.key_praise_quote}")
        if analysis.community_sentiment_analysis.main_criticism:
            print(f"⚠️ Main Criticism: {analysis.community_sentiment_analysis.main_criticism}")

async def print_final_report(final_results: List[FinalAnalysis], total_usage: ModelUsageAsync) -> None:
    """Print the final investment report."""
    # Sort results by score
    sorted_results = sorted(final_results, key=lambda x: x.score or 0, reverse=True)
    
    print("\n\n" + "="*80)
    print("📈 FINAL VC INVESTMENT REPORT")
    print("="*80)
    
    for i, result in enumerate(sorted_results, 1):
        print(f"\n🏆 RANK #{i}")
        print_analysis_summary(result)
    
    print("\n" + "="*80)
    print("📊 PIPELINE STATISTICS")
    print("="*80)
    print(f"🔢 Total Repositories Analyzed: {len(final_results)}")
    print(f"✅ Successful Repo Rubric Analyses: {sum(1 for r in final_results if r.repo_rubric_analysis)}")
    print(f"😊 Successful Community Sentiment Analyses: {sum(1 for r in final_results if r.community_sentiment_analysis)}")
    print(f"💰 Analysis Completeness: {len([r for r in final_results if r.repo_rubric_analysis and r.community_sentiment_analysis])}/{len(final_results)} repositories fully analyzed")
    
    # Display cost information
    total_cost = await total_usage.get_cost()
    total_tokens = await total_usage.get_tokens_used()
    print(f"\n💸 Total Analysis Cost: ${total_cost:.4f}")
    print(f"🔢 Total Tokens Used: {total_tokens:,}")


In [137]:
def _is_classification_result(obj) -> TypeGuard[DevOpsClassification]:
    return not isinstance(obj, Exception) and obj is not None

# Production-Ready 3-Tier Filtering (Type-Safe)
async def filter_devops_repositories(
    period_trending_repos: Dict[str, dict],
    dev_ops_collections: List[Dict[str, str]],
    openai_client: AsyncOpenAI,
    github_client: Github,
    use_llm_classification: bool = True,
    fallback_to_keywords: bool = True,
    filter_big_tech_companies: bool = True,
    llm_batch_size: int = 10
) -> Dict[str, dict]:
    """Production-ready 3-tier filtering with proper error handling."""
    # Collect all repositories first
    all_repos = []
    for period, trending_data in period_trending_repos.items():
        if not trending_data or not trending_data.get("data"):
            continue
        all_repos.extend(trending_data["data"]["rows"])

    print(f"📊 Total repositories to process: {len(all_repos)}")

    # PHASE 0: Filter out big tech companies (PARALLEL VERSION)
    if filter_big_tech_companies:
        print(f"\n🏢 Phase 0 - Big Tech Filtering...")
        filtered_repos_list = []
        big_tech_usage = ModelUsageAsync()
        
        # Process in parallel batches for speed
        big_tech_batch_size = 10  # Same as LLM batch size
        total_batches = (len(all_repos) + big_tech_batch_size - 1) // big_tech_batch_size
        
        for batch_idx in range(0, len(all_repos), big_tech_batch_size):
            batch = all_repos[batch_idx:batch_idx + big_tech_batch_size]
            batch_num = (batch_idx // big_tech_batch_size) + 1
            
            print(f"🔄 Processing batch {batch_num}/{total_batches} ({len(batch)} repositories)")
            
            # Create parallel classification tasks
            classification_tasks = [
                classify_big_tech_company(
                    openai_client=openai_client,
                    github_client=github_client,
                    repo_name=repo["repo_name"],
                    description=repo.get("description", ""),
                    llm_usage=big_tech_usage
                )
                for repo in batch
            ]
            
            # Execute batch in parallel
            try:
                classifications = await asyncio.gather(*classification_tasks, return_exceptions=True)
                
                # Process results with explicit exception handling
                for repo, result in zip(batch, classifications):
                    # Handle exceptions explicitly
                    if isinstance(result, Exception):
                        print(f"⚠️ Big tech classification failed for {repo['repo_name']}: {result}")
                        filtered_repos_list.append(repo)  # Include on error (conservative)
                        continue
                    
                    # Process successful classification
                    if _is_classification_result(result) and result.is_big_tech:
                        print(f"🏢 Filtered out big tech: {repo['repo_name']} (confidence: {result.confidence_score}/5)")
                        # Skip this repo (don't add to filtered_repos_list)
                        continue
                    else:
                        filtered_repos_list.append(repo)
                        
            except Exception as e:
                print(f"⚠️ Batch processing failed: {e}")
                # Add all repos from failed batch (conservative approach)
                filtered_repos_list.extend(batch)
        
        # Show big tech filtering results
        big_tech_cost = await big_tech_usage.get_cost()
        print(f"💰 Big tech filtering cost: ${big_tech_cost:.4f}")
        print(f"📊 Filtered out {len(all_repos) - len(filtered_repos_list)} big tech repos")
        print(f"📊 Remaining repos for DevOps filtering: {len(filtered_repos_list)}")
        
        # Reconstruct period_trending_repos with filtered repos for processing
        period_trending_repos = {
            "filtered": {
                "data": {
                    "rows": filtered_repos_list
                }
            }
        }
    
    # TIER 1: Collection-based filtering
    dev_ops_collection_names = {
        name for collection_dict in dev_ops_collections 
        for name in collection_dict.values()
    }
    
    potential_leads = {}
    collection_matches = 0
    llm_matches = 0
    keyword_matches = 0
    
    # DevOps keywords for final fallback
    devops_keywords = {
        'docker', 'kubernetes', 'k8s', 'terraform', 'ansible', 'jenkins', 
        'ci/cd', 'cicd', 'deployment', 'infrastructure', 'devops', 'monitoring',
        'prometheus', 'grafana', 'elasticsearch', 'nginx', 'apache', 'redis',
        'postgresql', 'mysql', 'microservices', 'container', 'orchestration',
        'automation', 'pipeline', 'cloud', 'aws', 'azure', 'gcp', 'helm'
    }
    
    print(f"🔍 3-Tier Filtering Strategy: Collections → LLM → Keywords")
    print(f"📋 Using {len(dev_ops_collection_names)} DevOps collections for Tier 1...")
    
    # Collect repositories for processing
    unmatched_repos = []
    
    for period, trending_data in period_trending_repos.items():
        if not trending_data or not trending_data.get("data"):
            continue
            
        print(f"📅 Processing {period}...")
        
        for repo in trending_data["data"]["rows"]:
            repo_name = repo["repo_name"]
            repo_collections = set(repo.get("collection_names", "").split(','))
            repo_collections = {c.strip() for c in repo_collections if c.strip()}
            
            # TIER 1: Collection-based filtering
            if not dev_ops_collection_names.isdisjoint(repo_collections):
                if repo_name not in potential_leads:
                    potential_leads[repo_name] = repo
                    matching_collections = repo_collections.intersection(dev_ops_collection_names)
                    print(f"✅ Found DevOps repo: {repo_name} (matches: {matching_collections})")
                    collection_matches += 1
                    continue
            
            # Store for LLM analysis
            if repo_name not in potential_leads:
                unmatched_repos.append(repo)
    
        # TIER 2: LLM-based classification (PARALLEL for speed)
        if use_llm_classification and unmatched_repos:
            print(f"\n🤖 Tier 2 - LLM Classification on {len(unmatched_repos)} repositories...")
            llm_usage = ModelUsageAsync()
            
            # Process in parallel batches for speed
            total_batches = (len(unmatched_repos) + llm_batch_size - 1) // llm_batch_size
            
            for batch_idx in range(0, len(unmatched_repos), llm_batch_size):
                batch = unmatched_repos[batch_idx:batch_idx + llm_batch_size]
                batch_num = (batch_idx // llm_batch_size) + 1
                
                print(f"🔄 Processing batch {batch_num}/{total_batches} ({len(batch)} repositories)")
                
                # Create parallel classification tasks
                classification_tasks = [
                    classify_repository_with_llm(
                        openai_client=openai_client,
                        repo_name=repo["repo_name"],
                        description=repo.get("description", ""),
                        llm_usage=llm_usage
                    )
                    for repo in batch if repo["repo_name"] not in potential_leads
                ]
                
                # Execute batch in parallel
                try:
                    classifications = await asyncio.gather(*classification_tasks, return_exceptions=True)
                    
                    # Process results with explicit type handling
                    repos_in_batch = [repo for repo in batch if repo["repo_name"] not in potential_leads]
                    
                    for repo, result in zip(repos_in_batch, classifications):
                        # Handle exceptions explicitly
                        if isinstance(result, Exception):
                            print(f"⚠️ LLM classification failed for {repo['repo_name']}: {result}")
                            continue
                        
                        # Type-safe processing: result is now Optional[DevOpsClassification]
                        if _is_classification_result(result):
                            try:
                                if result.is_devops_related:
                                    repo_name = repo["repo_name"]
                                    potential_leads[repo_name] = repo
                                    categories_str = ', '.join(result.devops_categories[:2]) if result.devops_categories else 'General DevOps'
                                    print(f"🧠 Tier 2 - LLM match: {repo_name} (confidence: {result.confidence_score}/5, categories: {categories_str})")
                                    llm_matches += 1

                            except AttributeError:
                                # Additional safety for any attribute access issues
                                print(f"⚠️ Invalid classification result for {repo['repo_name']}")
                                
                except Exception as e:
                    print(f"⚠️ Batch processing failed: {e}")
            
            # Show LLM costs
            llm_cost = await llm_usage.get_cost()
            print(f"💰 LLM classification cost: ${llm_cost:.4f}")
    
    # TIER 3: Keyword-based final fallback
    if fallback_to_keywords:
        print(f"\n🔍 Tier 3 - Keyword-based fallback...")
        for repo in unmatched_repos:
            repo_name = repo["repo_name"]
            if repo_name in potential_leads:
                continue  # Already matched
                
            description = repo.get("description", "").lower()
            repo_name_lower = repo_name.lower()
            
            # Check for DevOps keywords
            found_keywords = []
            for keyword in devops_keywords:
                if keyword in description or keyword in repo_name_lower:
                    found_keywords.append(keyword)
            
            if found_keywords:
                potential_leads[repo_name] = repo
                print(f"🎯 Tier 3 - Keyword match: {repo_name}")
                keyword_matches += 1
    
    print(f"\n📊 3-Tier Filtering Results:")
    print(f"   • Tier 1 (Collections): {collection_matches}")
    print(f"   • Tier 2 (LLM): {llm_matches}")
    print(f"   • Tier 3 (Keywords): {keyword_matches}")
    print(f"   • Total unique leads: {len(potential_leads)}")
    
    return potential_leads

print("✅ Production-ready 3-tier filtering function loaded")


✅ Production-ready 3-tier filtering function loaded


In [138]:
# Main VC Sourcing Pipeline Execution with 3-Tier Filtering
async def run_vc_sourcing_pipeline():
    """Execute the complete VC sourcing pipeline with enhanced 3-tier filtering."""
    print("🚀 Starting VC Sourcing Pipeline...")

    # Fetch Trending Repositories Data
    print("🚀 Phase 0: Fetching trending repositories data...")

    async with aiohttp.ClientSession() as session:
        period_trending_repos = {}
        for period in PERIOD_VALUES:
            print(f"📅 Fetching trending repos for {period}...")
            trending_data = await make_oss_insight_request(
                session=session,
                url=TRENDING_REPOS_URL,
                params={"period": period}
            )
            if trending_data:
                period_trending_repos[period] = trending_data
                repo_count = len(trending_data.get("data", {}).get("rows", []))
                print(f"✅ Found {repo_count} trending repositories for {period}")
            else:
                print(f"❌ Failed to fetch data for {period}")

        total_repos = sum(len(data.get("data", {}).get("rows", [])) for data in period_trending_repos.values())
        print(f"\n📊 Total repositories fetched: {total_repos} across {len(period_trending_repos)} periods")

        # Fetch Collections and Identify DevOps Collections
        print("🔍 Fetching and analyzing repository collections...")

        # Initialize cost tracking for DevOps collections identification
        dev_ops_collections_usage = ModelUsageAsync()
    
        collections_data = await make_oss_insight_request(
            session=session,
            url=COLLECTIONS_URL,
            params={}
        )

        if collections_data:
            total_collections = len(collections_data.get("data", {}).get("rows", []))
            print(f"✅ Found {total_collections} total collections")
            
            # Identify DevOps collections using LLM
            dev_ops_collections = await get_dev_ops_collections(
                openai_client=openai_client,
                collections_data=collections_data,
                llm_usage=dev_ops_collections_usage
            )
            
            if dev_ops_collections:
                dev_ops_collections = [{c.id: c.name} for c in dev_ops_collections]
                print(f"📋 DevOps collections identified:")
                for collection in dev_ops_collections[:10]:  # Show first 10
                    for id_key, name in collection.items():
                        print(f"  • {name} (ID: {id_key})")
                if len(dev_ops_collections) > 10:
                    print(f"  ... and {len(dev_ops_collections) - 10} more")
            
            # Display cost information
            cost = await dev_ops_collections_usage.get_cost()
            tokens = await dev_ops_collections_usage.get_tokens_used()
            print(f"💰 DevOps identification cost: ${cost:.4f} ({tokens:,} tokens)")
        else:
            print("❌ Failed to fetch collections data")
            dev_ops_collections = []

    # Data Collection Summary
    print("📊 Data Collection Phase Complete")
    print("=" * 50)

    # Summary of collected data
    if 'period_trending_repos' in globals():
        print(f"✅ Trending Repositories: {len(period_trending_repos)} time periods")
        for period, data in period_trending_repos.items():
            if data:
                count = len(data.get("data", {}).get("rows", []))
                print(f"   • {period}: {count} repositories")

    if 'dev_ops_collections' in globals():
        print(f"✅ DevOps Collections: {len(dev_ops_collections)} identified")
        
    print("\n🎯 Ready to proceed with VC sourcing pipeline!")
    print("📋 Data available for analysis:")
    print("   • Trending repository data across multiple time periods")
    print("   • DevOps-specific collection classifications")
    print("   • API clients configured for GitHub, Exa, and OpenAI")
    
    # Phase 1: Filter DevOps repositories from trending data

    # TODO: for demo, just use the repos in past month
    past_month_repos = period_trending_repos["past_month"]
    test_repos = {
        "past_month": past_month_repos
    }
    
    print("🔍 Using 3-Tier Enhanced Filtering: Collections → LLM → Keywords")
    potential_leads = await filter_devops_repositories(
        period_trending_repos=test_repos,
        dev_ops_collections=dev_ops_collections,
        openai_client=openai_client,
        github_client=github_client,
        use_llm_classification=True,
        fallback_to_keywords=True,
        filter_big_tech_companies=True,
        llm_batch_size=10,
    )
    
    if not potential_leads:
        print("❌ No DevOps repositories found. Pipeline ended.")
        return
    
    # Initialize usage tracker for the entire analysis run
    total_usage = ModelUsageAsync()
    final_results = []
    
    # TODO: for demo, just analyze first 10 repos instead of the entirety of potential_leads
    print(f"\n🔬 Phase 2 & 3: Analyzing 10 repositories...")
    
    # Limit analysis to first 10 repositories for demonstration
    # Remove this limit for full analysis
    limited_leads = dict(list(potential_leads.items())[:10])
    print(f"📝 Note: Analyzing first {len(limited_leads)} repositories for demonstration")
    
    # Analyze each repository
    for repo_name, repo_data in limited_leads.items():
        try:
            analysis_result = await analyze_repository_pipeline(
                repo_name=repo_name,
                repo_data=repo_data,
                openai_client=openai_client,
                github_client=github_client,
                exa_client=exa_client,
                llm_usage=total_usage
            )
            final_results.append(analysis_result)
            print(f"✅ Completed analysis for {repo_name}")
            
        except Exception as e:
            print(f"❌ Failed to analyze {repo_name}: {e}")
    
    # Phase 4: Generate final report
    print(f"\n🏁 Phase 4: Scoring and Reporting...")
    await print_final_report(final_results, total_usage)
    
    return final_results

print("✅ VC Sourcing Pipeline functions loaded")
print("📋 Ready to execute pipeline - run the next cell to start analysis")


✅ VC Sourcing Pipeline functions loaded
📋 Ready to execute pipeline - run the next cell to start analysis


In [139]:
# Execute with 3-Tier Enhanced Filtering
# await run_vc_sourcing_pipeline()

# Uploading Results to DynamoDB

In [140]:
# Add to your existing imports
import boto3  # type: ignore
import json
from decimal import Decimal
from datetime import datetime
from typing import Any
from botocore.exceptions import ClientError # type: ignore

In [141]:
# Initialize DynamoDB client
# Create session with default profile
session = boto3.Session(profile_name='root')  # or your profile name

dynamodb = session.resource('dynamodb')
dynamodb_client = session.client('dynamodb')

# # Simplified DynamoDB Table Creation (No GSIs)
# def create_vc_analysis_table(table_name: str = "vc-sourcing-analysis") -> bool:
#     """Create DynamoDB table for VC analysis results with composite key."""
#     try:
#         table = dynamodb.create_table(
#             TableName=table_name,
#             KeySchema=[
#                 {
#                     'AttributeName': 'repo_name',
#                     'KeyType': 'HASH'  # Primary key
#                 },
#                 {
#                     'AttributeName': 'analysis_date',
#                     'KeyType': 'RANGE'  # Sort key
#                 }
#             ],
#             AttributeDefinitions=[
#                 {
#                     'AttributeName': 'repo_name',
#                     'AttributeType': 'S'
#                 },
#                 {
#                     'AttributeName': 'analysis_date',
#                     'AttributeType': 'S'
#                 }
#             ],
#             BillingMode='PAY_PER_REQUEST'  # On-demand pricing
#         )
        
#         # Wait for table to be created
#         print(f"🏗️ Creating DynamoDB table '{table_name}'...")
#         table.wait_until_exists()
#         print(f"✅ Table '{table_name}' created successfully!")
#         return True
        
#     except ClientError as e:
#         if e.response['Error']['Code'] == 'ResourceInUseException':
#             print(f"📋 Table '{table_name}' already exists")
#             return True
#         else:
#             print(f"❌ Error creating table: {e}")
#             return False

# create_vc_analysis_table()

In [ ]:
def convert_to_dynamodb_format(obj: Any) -> Any:  # Changed return type
    """Convert Python objects to DynamoDB compatible format."""
    if isinstance(obj, dict):
        return {k: convert_to_dynamodb_format(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_dynamodb_format(item) for item in obj]
    elif isinstance(obj, float):
        return Decimal(str(obj))
    elif isinstance(obj, int):
        return Decimal(str(obj))
    elif obj is None:
        return None
    else:
        return obj

def serialize_analysis_for_dynamodb(analysis: FinalAnalysis) -> dict[str, Any]:  # More specific return type
    """Convert FinalAnalysis object to DynamoDB-compatible format."""
    
    # Convert Pydantic models to dictionaries
    item: dict[str, Any] = {  # Explicit type annotation
        'repo_name': analysis.repo_name,
        'analysis_date': datetime.now().strftime('%Y-%m-%d'),
        'final_score': Decimal(str(analysis.score)) if analysis.score else Decimal('0'),
        
        # OSS Insight Data
        'oss_insight_data': convert_to_dynamodb_format(analysis.oss_insight_data),
        
        # Repository Analysis
        'repo_analysis': None,
        'community_analysis': None
    }
    
    # Add repo rubric analysis if available
    if analysis.repo_rubric_analysis:
        item['repo_analysis'] = {
            'problem_clarity_score': Decimal(str(analysis.repo_rubric_analysis.problem_clarity_score)),
            'adoption_ease_score': Decimal(str(analysis.repo_rubric_analysis.adoption_ease_score)),
            'maturity_health_score': Decimal(str(analysis.repo_rubric_analysis.maturity_health_score)),
            'problem_solved': analysis.repo_rubric_analysis.problem_solved
        }
    
    # Add community sentiment analysis if available  
    if analysis.community_sentiment_analysis:
        item['community_analysis'] = {
            'excitement_score': Decimal(str(analysis.community_sentiment_analysis.excitement_score)),
            'problem_solution_fit_score': Decimal(str(analysis.community_sentiment_analysis.problem_solution_fit_score)),
            'credibility_adoption_score': Decimal(str(analysis.community_sentiment_analysis.credibility_adoption_score)),
            'key_praise_quote': analysis.community_sentiment_analysis.key_praise_quote,
            'main_criticism': analysis.community_sentiment_analysis.main_criticism
        }
    
    return item

async def upload_analysis_to_dynamodb(
    final_results: List[FinalAnalysis], 
    table_name: str = "vc-sourcing-analysis"
) -> bool:
    """Upload analysis results to DynamoDB table."""
    
    try:
        table = dynamodb.Table(table_name)
        
        print(f"📤 Uploading {len(final_results)} analysis results to DynamoDB...")
        
        # Upload each analysis result
        successful_uploads = 0
        failed_uploads = 0
        
        for analysis in final_results:
            try:
                item = serialize_analysis_for_dynamodb(analysis)
                
                # Put item in DynamoDB
                table.put_item(Item=item)
                
                print(f"✅ Uploaded: {analysis.repo_name} (Score: {analysis.score})")
                successful_uploads += 1
                
            except Exception as e:
                print(f"❌ Failed to upload {analysis.repo_name}: {e}")
                failed_uploads += 1
        
        print(f"\n📊 Upload Summary:")
        print(f"   ✅ Successful: {successful_uploads}")
        print(f"   ❌ Failed: {failed_uploads}")
        print(f"   📋 Total: {len(final_results)}")
        
        return failed_uploads == 0
        
    except Exception as e:
        print(f"❌ DynamoDB upload error: {e}")
        return False

def query_top_repos_by_score(table_name: str = "vc-sourcing-analysis", limit: int = 10) -> List[dict[str, Any]]:
    """Query top repositories by score from DynamoDB."""
    try:
        table = dynamodb.Table(table_name)
        
        # Scan table and sort by score (since we can't easily query by score range)
        response = table.scan()
        
        items = response['Items']
        
        # Sort by final_score in descending order
        sorted_items = sorted(items, key=lambda x: float(x.get('final_score', 0)), reverse=True)
        
        return sorted_items[:limit]
        
    except Exception as e:
        print(f"❌ Error querying DynamoDB: {e}")
        return []

def print_dynamodb_results(items: List[dict[str, Any]]) -> None:
    """Print formatted results from DynamoDB query."""
    print("\n🏆 TOP REPOSITORIES FROM DYNAMODB:")
    print("=" * 60)
    
    for i, item in enumerate(items, 1):
        repo_name = item.get('repo_name', 'Unknown')
        score = float(item.get('final_score', 0))
        analysis_date = item.get('analysis_date', 'Unknown')
        
        # Get OSS Insight data
        oss_data = item.get('oss_insight_data', {})
        total_score = oss_data.get('total_score', 'N/A')
        stars = oss_data.get('stars', 'N/A')
        description = oss_data.get('description', 'No description available')
        
        print(f"\n## {repo_name} | 📊 Score: {score}")
        print(f"📈 Total Score: {total_score} | ⭐ Stars: {stars}")
        print(f"📝 Description: {description}")
        
        # Show repo analysis details if available
        if item.get('repo_analysis'):
            repo_analysis = item['repo_analysis']
            problem_solved = repo_analysis.get('problem_solved', 'Problem description not available')
            print(f"📖 Problem Solved: {problem_solved}")
            print(f"🎯 Problem Statement Clarity: {repo_analysis.get('problem_clarity_score', 'N/A')}/5")
            print(f"⚡ Ease of Adoption: {repo_analysis.get('adoption_ease_score', 'N/A')}/5")
            print(f"💊 Project Maturity & Community Health: {repo_analysis.get('maturity_health_score', 'N/A')}/5")
        
        # Show community sentiment details if available
        if item.get('community_analysis'):
            community = item['community_analysis']
            print(f"😊 Community Sentiment: {community.get('excitement_score', 'N/A')}/5")
            print(f"📈 Problem Solution Fit: {community.get('problem_solution_fit_score', 'N/A')}/5")
            print(f"💊 Credibility & Adoption: {community.get('credibility_adoption_score', 'N/A')}/5")
            
            # Add quotes if available
            key_praise = community.get('key_praise_quote')
            main_criticism = community.get('main_criticism')
            
            if key_praise:
                print(f"💬 Key Praise Quote: {key_praise}")
            if main_criticism:
                print(f"⚠️ Main Criticism: {main_criticism}")

print("✅ Updated DynamoDB results display function")

✅ DynamoDB integration functions loaded


In [143]:
# Updated Main Pipeline with DynamoDB Integration
async def run_vc_sourcing_pipeline_with_dynamodb(table_name: str = "vc-sourcing-analysis"):
    """Execute VC sourcing pipeline and upload results to DynamoDB."""
    
    # Run your existing pipeline
    final_results = await run_vc_sourcing_pipeline()
    
    if not final_results:
        print("❌ No analysis results to upload")
        return
    
    # Upload results to DynamoDB
    print(f"\n💾 Phase 5: Uploading results to DynamoDB...")
    upload_success = await upload_analysis_to_dynamodb(final_results, table_name)
    
    if upload_success:
        print(f"✅ All results uploaded successfully to '{table_name}'")
        
        # Query and display top results from DynamoDB
        print(f"\n🔍 Querying top results from DynamoDB...")
        top_repos = query_top_repos_by_score(table_name, limit=5)
        print_dynamodb_results(top_repos)
        
    else:
        print(f"⚠️ Some uploads failed. Check logs above.")
    
    return final_results

print("✅ Updated pipeline with DynamoDB integration loaded")

✅ Updated pipeline with DynamoDB integration loaded


In [144]:
await run_vc_sourcing_pipeline_with_dynamodb()

🚀 Starting VC Sourcing Pipeline...
🚀 Phase 0: Fetching trending repositories data...
📅 Fetching trending repos for past_24_hours...
✅ Found 100 trending repositories for past_24_hours
📅 Fetching trending repos for past_week...
✅ Found 100 trending repositories for past_week
📅 Fetching trending repos for past_month...
✅ Found 100 trending repositories for past_month
📅 Fetching trending repos for past_3_months...
✅ Found 100 trending repositories for past_3_months

📊 Total repositories fetched: 400 across 4 periods
🔍 Fetching and analyzing repository collections...
✅ Found 95 total collections
🔍 Analyzing 95 collections to identify DevOps-related ones...
✅ Identified 62 DevOps collections
📋 DevOps collections identified:
  • Static Site Generator (ID: 1)
  • Open Source Database (ID: 2)
  • Web Framework (ID: 10004)
  • Business Intelligence (ID: 10006)
  • Time Series Database (ID: 10007)
  • Graph Database (ID: 10008)
  • Github Alternative (ID: 10009)
  • Artificial Intelligence (ID: 

[FinalAnalysis(repo_name='n8n-io/n8n', oss_insight_data={'repo_id': '193215554', 'repo_name': 'n8n-io/n8n', 'primary_language': 'TypeScript', 'description': 'Fair-code workflow automation platform with native AI capabilities. Combine visual building with custom code, self-host or cloud, 400+ integrations.', 'stars': '7482', 'forks': '3321', 'pull_requests': '242', 'pushes': '1001', 'total_score': '34714.0041', 'contributor_logins': 'shortstacked,tomi,ivov,mutdmour,autologie', 'collection_names': 'Low Code Development Tool,Zapier Alternatives'}, repo_rubric_analysis=RepoRubricAnalysis(problem_clarity_score=5, adoption_ease_score=5, maturity_health_score=5, problem_solved='It removes the friction of building and managing complex integrations by providing a self-hostable, code-extensible no-code platform with hundreds of prebuilt connectors.'), community_sentiment_analysis=CommunitySentimentAnalysis(excitement_score=4, problem_solution_fit_score=5, credibility_adoption_score=4, key_praise